In [3]:
matrix_1 = [[0,'A',  'B', 'C',  'D'],
          ['A',  0,   16,  16,  10], 
          ['B',  0,    0,   8,  8],
          ['C',  0,    0,   0,  4],
          ['D',  0,    0,   0,  0]]

matrix_2 = [[0,  'A', 'B', 'C', 'D', 'E', 'F'],
            ['A', 0,   5,   4,   7,   6,   8], 
            ['B', 0,   0,   7,  10,   9,  11],
            ['C', 0,   0,   0,   7,   6,   8],
            ['D', 0,   0,   0,   0,   5,   9],
            ['E', 0,   0,   0,   0,   0,   8],
            ['F', 0,   0,   0,   0,   0,   0]]

In [15]:
def toFixed(numObj, digits=0):
    return f"{numObj:.{digits}f}"

In [58]:
def NJ(matrix):
    n = len(matrix) - 1
    newick_dict = {}
    last = n-3
    for k in range(n-2):
        sum_dict = {matrix[i][0] : 0 for i in range(1, n+1)}
        add_dict = {}
        min_Q = float('inf')
        for i in range(1, n):
            for j in range(i+1, n+1):
                sum_dict[matrix[i][0]] += matrix[i][j]
                sum_dict[matrix[0][j]] += matrix[i][j]
        for i in range(1, n):
            for j in range(i+1, n+1):
                if ((n/(n-2))*matrix[i][j] - sum_dict[matrix[0][j]] - sum_dict[matrix[i][0]]) < min_Q:
                    min_Q = (n/(n-2))*matrix[i][j] - sum_dict[matrix[0][j]] - sum_dict[matrix[i][0]]
                    i_min, j_min = i, j
                elif ((n/(n-2))*matrix[i][j] - sum_dict[matrix[0][j]] - sum_dict[matrix[i][0]]) == min_Q:
                    if (abs(sum_dict[matrix[0][j]] - sum_dict[matrix[i][0]])*(-1) + matrix[i][j]) >= (abs(sum_dict[matrix[0][j_min]] - sum_dict[matrix[i_min][0]])*(-1) + matrix[i_min][j_min]):
                        min_Q = (n/(n-2))*matrix[i][j] - sum_dict[matrix[0][j]] - sum_dict[matrix[i][0]]
                        i_min, j_min = i, j
        new_node = ''.join(sorted(matrix[i_min][0] + matrix[0][j_min]))
        d_1 = 0.5 * (matrix[i_min][j_min] + sum_dict[matrix[i_min][0]] - sum_dict[matrix[0][j_min]])
        d_2 = 0.5 * (matrix[i_min][j_min] - sum_dict[matrix[i_min][0]] + sum_dict[matrix[0][j_min]])
        if k != last:
            if (matrix[i_min][0] not in newick_dict) and (matrix[0][j_min] not in newick_dict):
                newick_dict[new_node] = '(' + matrix[i_min][0] + ':' + toFixed(d_1, 2) + ',' + matrix[0][j_min] + ':' + toFixed(d_2, 2) + ')' 
            elif (matrix[i_min][0] not in newick_dict) and (matrix[0][j_min] in newick_dict):
                newick_dict[new_node] = '(' + matrix[i_min][0] + ':' + toFixed(d_1, 2) + ',' + newick_dict[matrix[0][j_min]] + ':' + toFixed(d_2, 2) + ')' 
            elif (matrix[i_min][0] in newick_dict) and (matrix[0][j_min] not in newick_dict):
                newick_dict[new_node] = '(' + newick_dict[matrix[i_min][0]] + ':' + toFixed(d_1, 2) + ',' + matrix[0][j_min] + ':' + toFixed(d_2, 2) + ')'
            elif (matrix[i_min][0] in newick_dict) and (matrix[0][j_min] in newick_dict):
                newick_dict[new_node] = '(' + newick_dict[matrix[i_min][0]] + ':' + toFixed(d_1, 2) + ',' + newick_dict[matrix[0][j_min]] + ':' + toFixed(d_2, 2) + ')'
        new_matrix = [[0 for t in range(n)] for t in range(n)]
        new_matrix[0][1], new_matrix[1][0] = new_node, new_node
        i = 2
        for j in range(1, n+1):
            if matrix[0][j][0] not in new_matrix[0][1]:
                new_matrix[0][i] = matrix[0][j]
                new_matrix[i][0] = matrix[0][j]
                i += 1
        for i in range(1, n):
            for j in range(i+1, n+1):
                if ((matrix[0][j][0] not in new_node) and  (matrix[i][0][0] in new_node)) and  (''.join(sorted(matrix[0][j] + new_node)) not in add_dict)  :
                    add_dict[''.join(sorted(matrix[0][j] + new_node))] = matrix[i][j]
                elif ((matrix[0][j][0] not in new_node) and  (matrix[i][0][0] in new_node)) and  (''.join(sorted(matrix[0][j] + new_node)) in add_dict):
                    add_dict[''.join(sorted(matrix[0][j] + new_node))] += matrix[i][j]
                elif ((matrix[0][j][0] in new_node) and  (matrix[i][0][0] not in new_node)) and  (''.join(sorted(matrix[i][0] + new_node)) not in add_dict):
                    add_dict[''.join(sorted(matrix[i][0] + new_node))] = matrix[i][j]
                elif ((matrix[0][j][0] in new_node) and  (matrix[i][0][0] not in new_node)) and  (''.join(sorted(matrix[i][0] + new_node)) in add_dict):
                    add_dict[''.join(sorted(matrix[i][0] + new_node))] += matrix[i][j]
                elif ((matrix[0][j][0] not in new_node) and  (matrix[i][0][0] not in new_node)):
                    add_dict[''.join(sorted(matrix[i][0] + matrix[0][j]))] = matrix[i][j]
        for i in range(1,n-1):
            for j in range(i+1, n):
                if ((new_matrix[0][j][0] not in new_node) and  (new_matrix[i][0][0] not in new_node)):
                    new_matrix[i][j] = add_dict[''.join(sorted(new_matrix[i][0] + new_matrix[0][j]))]
                else:
                    new_matrix[i][j] =+ 0.5 * (add_dict[''.join(sorted(new_matrix[i][0] + new_matrix[0][j]))] - matrix[i_min][j_min])
        if k == last:
            if new_matrix[0][2] not in newick_dict:
                if (matrix[i_min][0] not in newick_dict) and (matrix[0][j_min] not in newick_dict):
                    newick_dict[new_node] = '(' + matrix[i_min][0] + ':' + toFixed(d_1, 2) + ',' + matrix[0][j_min] + ':' + toFixed(d_2, 2) + ',' + new_matrix[0][2] + ':' + toFixed(new_matrix[1][2], 2) + ')' 
                elif (matrix[i_min][0] not in newick_dict) and (matrix[0][j_min] in newick_dict):
                    newick_dict[new_node] = '(' + matrix[i_min][0] + ':' + toFixed(d_1, 2) + ',' + newick_dict[matrix[0][j_min]] + ':' + toFixed(d_2, 2) + ',' + new_matrix[0][2] + ':' + toFixed(new_matrix[1][2], 2) + ')'
                elif (matrix[i_min][0] in newick_dict) and (matrix[0][j_min] not in newick_dict):
                    newick_dict[new_node] = '(' + newick_dict[matrix[i_min][0]] + ':' + toFixed(d_1, 2) + ',' + matrix[0][j_min] + ':' + toFixed(d_2, 2) + ',' + new_matrix[0][2] + ':' + toFixed(new_matrix[1][2], 2) + ')'
                elif (matrix[i_min][0] in newick_dict) and (matrix[0][j_min] in newick_dict):
                    newick_dict[new_node] = '(' + newick_dict[matrix[i_min][0]] + ':' + toFixed(d_1, 2) + ',' + newick_dict[matrix[0][j_min]] + ':' + toFixed(d_2, 2) + ',' + new_matrix[0][2] + ':' + toFixed(new_matrix[1][2], 2) + ')'
            else:
                if (matrix[i_min][0] not in newick_dict) and (matrix[0][j_min] not in newick_dict):
                    newick_dict[new_node] = '(' + matrix[i_min][0] + ':' + toFixed(d_1, 2) + ',' + matrix[0][j_min] + ':' + toFixed(d_2, 2) + ',' + newick_dict[new_matrix[0][2]] + ':' + toFixed(new_matrix[1][2], 2) + ')' 
                elif (matrix[i_min][0] not in newick_dict) and (matrix[0][j_min] in newick_dict):
                    newick_dict[new_node] = '(' + matrix[i_min][0] + ':' + toFixed(d_1, 2) + ',' + newick_dict[matrix[0][j_min]] + ':' + toFixed(d_2, 2) + ',' + newick_dict[new_matrix[0][2]] + ':' + toFixed(new_matrix[1][2], 2) + ')'
                elif (matrix[i_min][0] in newick_dict) and (matrix[0][j_min] not in newick_dict):
                    newick_dict[new_node] = '(' + newick_dict[matrix[i_min][0]] + ':' + toFixed(d_1, 2) + ',' + matrix[0][j_min] + ':' + toFixed(d_2, 2) + ',' + newick_dict[new_matrix[0][2]] + ':' + toFixed(new_matrix[1][2], 2) + ')'
                elif (matrix[i_min][0] in newick_dict) and (matrix[0][j_min] in newick_dict):
                    newick_dict[new_node] = '(' + newick_dict[matrix[i_min][0]] + ':' + toFixed(d_1, 2) + ',' + newick_dict[matrix[0][j_min]] + ':' + toFixed(d_2, 2) + ',' + newick_dict[new_matrix[0][2]] + ':' + toFixed(new_matrix[1][2], 2) + ')'
        n -= 1
        matrix = new_matrix
        min_Q = float('inf')
        i_min, j_min = 0, 0
    return(newick_dict[new_node])
NJ(matrix_2)

'((A:1.50,C:2.50):0.25,(B:4.50,F:6.50):-0.75,(D:4.00,E:1.00):1.75)'

In [ ]:
#(((D:4.00,E:1.00):1.50,C:2.50):0.25,(B:4.50,F:6.50):-0.75,A:1.75) - дерево получается при слиянии вершин DE и C c min_Q = -11

#((A:1.50,C:2.50):0.25,(B:4.50,F:6.50):-0.75,(D:4.00,E:1.00):1.75) - дерево получается при слиянии вершин A и C c min_Q = -11

# В обоих случаях содержится отрицательное ребро